# Imports

In [1]:
import dlib,cv2
import numpy as np
from facePoints import facePoints
import math
from scipy import ndimage
from copy import deepcopy
import os
from matplotlib import pyplot as plt


# Denoise

In [2]:
subjects = os.listdir('datasets/CASME2/Cropped-updated/Cropped/')
new_subdir = 'denoised/'

try:

    dest_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/'+new_subdir 
    os.mkdir(dest_subjectfolder)
except:
    pass


for subject in subjects:
    #create folder
    source_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/'+subject +'/'

    if subject=='masked' or subject=='masked_denoised' or subject=='denoised' or subject=='denoised_masked':
        continue
    try:

        dest_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/'+new_subdir + subject +'/'
        os.mkdir(dest_subjectfolder)
    except:
        pass

    source_sample_folder_list = os.listdir(source_subjectfolder) 

    for source_sample_folder in source_sample_folder_list:
        source_samplepath = source_subjectfolder + source_sample_folder +'/'
        try:
            dest_sample_folder = dest_subjectfolder + source_sample_folder +'/'
            os.mkdir(dest_sample_folder)
        except:
            pass    

        all_images = os.listdir(source_samplepath)
        all_images.sort()

        biggest_area_L = 0
        axesLength_opt_L = None
        center_coordinates_opt_L = None

        biggest_area_R = 0
        axesLength_opt_R = None
        center_coordinates_opt_R = None


        all_images_resized = [cv2.resize(cv2.imread(source_samplepath+ animage), (224,224)) for animage in all_images]
   
        #pad for noise calculation
        all_images_resized.insert(0, all_images_resized[1])
        all_images_resized.insert(0, all_images_resized[3])
        #index0 has become index2

        all_images_resized.append( all_images_resized[-1])
        all_images_resized.append( all_images_resized[-3])

#         int 	imgToDenoiseIndex,
#         int 	temporalWindowSize,
#         float 	h = 3,
#         int 	templateWindowSize = 7,
#         int 	searchWindowSize = 21 

        #all_images_denoised = [cv2.fastNlMeansDenoisingColoredMulti(animage, 2, 3, None, 4, 7, 35) for animage in all_images_resized]
        all_images_denoised = []

        for animage_th in range(len(all_images)):
            denoised_frame = cv2.fastNlMeansDenoisingColoredMulti(all_images_resized, animage_th+2, 3, None, 4, 7, 21)
            all_images_denoised.append(denoised_frame)


        for animage_th in range(len(all_images)): #apply best mask
            filename= all_images[animage_th]
            processed_image = all_images_denoised[animage_th]


            outputNameofImage = dest_sample_folder + 'd_' + filename
            print("Saving output image to", outputNameofImage)
            cv2.imwrite(outputNameofImage, processed_image)



Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img26.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img27.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img28.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img29.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img30.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img31.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img32.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img33.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02_03/d_reg_img34.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised/sub13/EP02

# Mask

In [3]:


def find_landmarks(imageRGB, Model_PATH = "shape_predictor_68_face_landmarks.dat"):

    frontalFaceDetector = dlib.get_frontal_face_detector()
    # Now the dlip shape_predictor class will take model and with the help of that, it will show 
    faceLandmarkDetector = dlib.shape_predictor(Model_PATH)
   
   # Now this line will try to detect all faces in an image either 1 or 2 or more faces
    allFaces = frontalFaceDetector(imageRGB, 0)
    #print(allFaces)


    #print("List of all faces detected: ",len(allFaces))

    # List to store landmarks of all detected faces
    #allFacesLandmark = []

    # Below loop we will use to detect all faces one by one and apply landmarks on them

    # for k in range(0, len(allFaces)):
    # dlib rectangle class will detecting face so that landmark can apply inside of that area
    # faceRectangleDlib = dlib.rectangle(int(allFaces[k].left()),int(allFaces[k].top()),
    #     int(allFaces[k].right()),int(allFaces[k].bottom()))
    faceRectangleDlib = dlib.rectangle(int(0),int(0),
        int(imageRGB.shape[1]),int(imageRGB.shape[0]))
        # Now we are running loop on every detected face and putting landmark on that with the help of faceLandmarkDetector
    detectedLandmarks = faceLandmarkDetector(imageRGB, faceRectangleDlib)
        
        # count number of landmarks we actually detected on image
        # if k==0:
        #     print("Total number of face landmarks detected ",len(detectedLandmarks.parts()))

        # Svaing the landmark one by one to the output folder
        #allFacesLandmark.append(detectedLandmarks)


    # Now we are running loop on every detected face and putting landmark on that with the help of faceLandmarkDetector
    detectedLandmarks = faceLandmarkDetector(imageRGB, faceRectangleDlib)

    return detectedLandmarks.parts(), [faceRectangleDlib.tl_corner(),faceRectangleDlib.br_corner()]

def rotate_image_by_innereye(Image_PATH = 'img5.jpg', Model_PATH = "shape_predictor_68_face_landmarks.dat"):

    # Now we are reading image using openCV
    img= cv2.imread(Image_PATH)
    imageRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    detectedLandmarks, _ = find_landmarks(imageRGB)
 
    # inner_eye_my_left = detectedLandmarks[40-1]
    # x1, y1 = inner_eye_my_left.x, inner_eye_my_left.y

    # inner_eye_my_right = detectedLandmarks[43-1]
    # x2, y2 = inner_eye_my_right.x, inner_eye_my_right.y

    inner_eye_my_left = detectedLandmarks[0]
    x1, y1 = inner_eye_my_left.x, inner_eye_my_left.y

    inner_eye_my_right = detectedLandmarks[16]
    x2, y2 = inner_eye_my_right.x, inner_eye_my_right.y
    #print(y1,y2)

    TL = (x2-x1)/(  (   ((y2-y1)**2) + ((x2-x1)**2))**0.5    )
    TR = (y1-y2)/(  (   ((y2-y1)**2) + ((x2-x1)**2))**0.5    )
    BL = (y2-y1)/(  (   ((y2-y1)**2) + ((x2-x1)**2))**0.5    )
    BR = (x2-x1)/(  (   ((y2-y1)**2) + ((x2-x1)**2))**0.5    )

    R = np.array([[TL, TR], [BL, BR]])
    Rm = R.T
    cosine = R[0][0]

    deg_to_rotate = math.degrees(math.cosh(cosine)) -90

        #rotation angle in degree
    rotated = ndimage.rotate(img, deg_to_rotate)
    

    return  rotated

def provide_mask_guide(image, rect, all_facial_points):

    nose = all_facial_points[33]
    left_eye_points = all_facial_points[36:42]
    right_eye_points = all_facial_points[42:48]
    left_eye_brow_points = all_facial_points[17:222]
    right_eye_brow_points = all_facial_points[22:27]
    jaw_points = all_facial_points[:17]
    y_size, x_size = image.shape[0], image.shape[1]

    out = {}
    out['jaw'] = [(max(0,a.x), min(max(0,a.y), y_size)) for a in jaw_points]
    out['left_eye'] = [(a.x, a.y) for a in left_eye_points]
    out['right_eye'] = [(a.x, a.y) for a in right_eye_points]

    return out

def crop_image(image, rect, all_facial_points, jaw_thicken = 5, addition = 5, top_allowance = 30, bottom_allowance = 10):
	# extract the starting and ending (x, y)-coordinates of the
	# bounding box
    nose = all_facial_points[33]
    left_eye_points = all_facial_points[36:42]
    right_eye_points = all_facial_points[42:48]
    left_eye_brow_points = all_facial_points[17:222]
    right_eye_brow_points = all_facial_points[22:27]
    jaw_points = all_facial_points[:17]

    startX = rect[0].x
    startY = rect[0].y
    endX = rect[1].x
    endY = rect[1].y


    #find top line
    startY = min ( min([pt.y for pt in left_eye_brow_points]),  min([pt.y for pt in right_eye_brow_points]))  - top_allowance
    endY = max([pt.y for pt in jaw_points]) + bottom_allowance

    #horizontal adjust frame to centre face
    left_to_centre = nose.x - startX
    centre_to_right = endX - nose.x
    max_dist = max(left_to_centre, centre_to_right)
    startX = nose.x  - max_dist
    endX = nose.x  + max_dist

    len_y = endY-startY
    len_x = endX- startX

    # #vertically adjust frame to centre face
    # top_to_centre = nose.y - startY
    # centre_to_bot = endY - nose.y
    # max_dist = max(top_to_centre, centre_to_bot)
    # startY = nose.y  - max_dist
    # endY = nose.y  + max_dist

    # len_y = endY-startY
    # len_x = endX- startX


    #jaw points
    adjusted_jaw_points = []

    counter = 0

    for apoint in jaw_points:

        x, y = apoint.x, apoint.y

        
        #print(x, endX)
        while x < 0:
            x+=addition
            len_x+=addition
            startX-=addition
            adjusted_jaw_points = [(coord[0]+addition, coord[1]) for coord in adjusted_jaw_points]

        while x > endX:
            endX+=addition
            len_x+=addition

        while y < 0:
            y+addition
            len_y+=addition
            startY-addition
            adjusted_jaw_points = [(coord[0], coord[1]+addition) for coord in adjusted_jaw_points]

        while y > endY:
            endY+=addition


        if x<(image.shape[0])/2:
            x+=jaw_thicken
        else:
            x-=jaw_thicken

        y-=jaw_thicken
        
        x -= startX
        y -= startY

        if counter == 0:
            reference1 = x

        if counter == len(jaw_points)-1:
            reference2 = x
            
        adjusted_jaw_points.append((x,y))
        counter+=1
        
    len_y = endY-startY
    len_x = endX- startX

    #adjusted_jaw_points.insert(0,(reference1,0))
    adjusted_jaw_points.insert(0,(0,0))
    adjusted_jaw_points.insert(0,(0, len_y))

    #adjusted_jaw_points.append((reference2,0))
    adjusted_jaw_points.append((len_x,0))
    adjusted_jaw_points.append((len_x, len_y))

   #left eye points
    adjusted_left_eye_points = []
    for apoint in left_eye_points:

        x, y = apoint.x, apoint.y
        x -= startX
        y -= startY

        if (x>0 and y>0) and (x<endX and y<endY):
            adjusted_left_eye_points.append((x,y))
            
    #right eye points
    adjusted_right_eye_points = []
    for apoint in right_eye_points:

        x, y = apoint.x, apoint.y
        x -= startX
        y -= startY

        if (x>0 and y>0) and (x<endX and y<endY):
            adjusted_right_eye_points.append((x,y))
    
    mask_guides = {}
    mask_guides['jaw'] = adjusted_jaw_points
    mask_guides['left_eye'] = adjusted_left_eye_points
    mask_guides['right_eye'] = adjusted_right_eye_points


    return  image[startY:endY, startX:endX], mask_guides 

def find_eye_maskarea_axeslength_centrecoord(cropped_img, mask_guides, eye):
    key = eye + '_eye'
    left_eye_end1 = mask_guides[key][0][0]
    left_eye_end2 = mask_guides[key][3][0]

    if mask_guides[key][1][1]<mask_guides[key][2][1]:
        left_eye_T = mask_guides[key][1][1]
    else:
        left_eye_T = mask_guides[key][2][1]

    if mask_guides[key][4][1]<mask_guides[key][5][1]:
        left_eye_B = mask_guides[key][5][1]
    else:
        left_eye_B = mask_guides[key][4][1]
    center_coordinates =  (int((left_eye_end1+left_eye_end2)/2) , int((left_eye_T+left_eye_B)/2))
    axesLength = ( int(abs(left_eye_end2 -left_eye_end1 )/1.2), int(abs(left_eye_B -left_eye_T)/0.8))
    est_area_mask = axesLength[0] * axesLength[1]
    return est_area_mask, axesLength, center_coordinates


def draw_eye_mask(image, axesLength, center_coordinates):
    color = (0, 0, 0)

    angle = 0
    
    startAngle = 0
    
    endAngle = 360

    thickness = -1
    todraw = deepcopy(image)

    masked = cv2.ellipse(todraw, center_coordinates, axesLength, angle,
                            startAngle, endAngle, color, thickness)
    return masked

def draw_jaw_mask(img , mask_guides, border=5):
    jaw_guide = mask_guides['jaw']



    jaw_guide = [(a[0], a[1]-border )for a in jaw_guide ]
    jaw_guide.insert(0, (0,img.shape[0]))
    jaw_guide.append((img.shape[1],img.shape[0]))
    points = np.array(jaw_guide)
    
    
    # Use fillPoly() function and give input as
    # image, end points,color of polygon
    # Here color of polygon will blue
    out = deepcopy(img)
    return cv2.fillPoly(out, pts=[points], color=(0, 0, 0))

In [4]:
def make_mask(source_sample_folder, animage, dest_sample_folder, ):
    # Loop through all directories and 
    # select images to crop using onset and apex frame distance
    ##https://pyimagesearch.com/2017/05/22/face-alignment-with-opencv-and-python/ TRY THIS FOR ROTATION TO SOLVE img5
    rotated_img = cv2.imread(source_sample_folder+ animage)
    all_facial_points, rectangle = find_landmarks(rotated_img)

    # Save all cropped images into an intermediate folder, save mask_guides in dictionary subject:sample:frame:mask_guides
    mask_guides = provide_mask_guide(rotated_img,rectangle, all_facial_points)
    cropped_img = rotated_img

    # For each eye
    #  Keep dictionary of est_area_mask and axesLength for each subject's sample as 2d list --> subject:sample:eye:[[est_area_mask1,axesLength1 ], [est_area_mask2,axesLength2 ], ...]
    #   for sample, use the axesLength with biggest est_area_mask as chosen to mask using instance specific centre coordinates
    #!! CALC angle of eye slid
    est_area_mask, axesLength, center_coordinates = find_eye_maskarea_axeslength_centrecoord(cropped_img, mask_guides, 'left')
    masked_image1= draw_eye_mask(cropped_img, axesLength, center_coordinates)


    est_area_mask, axesLength, center_coordinates = find_eye_maskarea_axeslength_centrecoord(cropped_img, mask_guides, 'right')
    masked_image2= draw_eye_mask(masked_image1, axesLength, center_coordinates)

    # #save masked image in new folder
    masked_image3 = draw_jaw_mask(masked_image2 , mask_guides)



    outputNameofImage = dest_sample_folder + 'masked_' + animage
    print("Saving output image to", outputNameofImage)
    cv2.imwrite(outputNameofImage, masked_image3)



In [5]:
subjects = os.listdir('datasets/CASME2/Cropped-updated/Cropped/denoised')
new_subdir = 'denoised_masked/'

try:

    dest_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/'+new_subdir 
    os.mkdir(dest_subjectfolder)
except:
    pass


for subject in subjects:
    #create folder
    source_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/denoised/'+subject +'/'
    try:
        if subject!='masked':
            dest_subjectfolder = 'datasets/CASME2/Cropped-updated/Cropped/'+new_subdir + subject +'/'
            os.mkdir(dest_subjectfolder)
    except:
        pass

    source_sample_folder_list = os.listdir(source_subjectfolder) 

    for source_sample_folder in source_sample_folder_list:
        source_samplepath = source_subjectfolder + source_sample_folder +'/'
        try:
            dest_sample_folder = dest_subjectfolder + source_sample_folder +'/'
            os.mkdir(dest_sample_folder)
        except:
            pass    

        all_images = os.listdir(source_samplepath)
        all_images.sort()

        biggest_area_L = 0
        axesLength_opt_L = None
        center_coordinates_opt_L = None

        biggest_area_R = 0
        axesLength_opt_R = None
        center_coordinates_opt_R = None



        for animage in all_images:
            # if 'masked_' + animage in os.listdir(dest_sample_folder):
            #     print('mask created: '+ dest_sample_folder + 'masked_' + animage)
            #     continue 
            #get best mask
            image_source = cv2.imread(source_samplepath+ animage)
            all_facial_points, rectangle = find_landmarks(image_source)

            # Save all cropped images into an intermediate folder, save mask_guides in dictionary subject:sample:frame:mask_guides
            mask_guides = provide_mask_guide(image_source,rectangle, all_facial_points)

            # For each eye
            #  Keep dictionary of est_area_mask and axesLength for each subject's sample as 2d list --> subject:sample:eye:[[est_area_mask1,axesLength1 ], [est_area_mask2,axesLength2 ], ...]
            #   for sample, use the axesLength with biggest est_area_mask as chosen to mask using instance specific centre coordinates
            #!! CALC angle of eye slid
            est_area_mask, axesLength, center_coordinates = find_eye_maskarea_axeslength_centrecoord(image_source, mask_guides, 'left')
            if est_area_mask>biggest_area_L:
                biggest_area_L = est_area_mask
                axesLength_opt_L = axesLength
                center_coordinates_opt_L = center_coordinates

            est_area_mask, axesLength, center_coordinates = find_eye_maskarea_axeslength_centrecoord(image_source, mask_guides, 'right')
            if est_area_mask>biggest_area_R:
                biggest_area_R = est_area_mask
                axesLength_opt_R = axesLength
                center_coordinates_opt_R = center_coordinates

        for animage in all_images: #apply best mask
            image_source = cv2.imread(source_samplepath+ animage)
            masked_image1= draw_eye_mask(image_source, axesLength_opt_L, center_coordinates_opt_L)


            masked_image2= draw_eye_mask(masked_image1, axesLength_opt_R, center_coordinates_opt_R)
            
            masked_image3 = draw_jaw_mask(masked_image2 , mask_guides)



            outputNameofImage = dest_sample_folder + 'm' + animage
            print("Saving output image to", outputNameofImage)
            cv2.imwrite(outputNameofImage, masked_image3)

Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img26.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img27.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img28.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img29.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img30.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img31.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img32.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img33.jpg
Saving output image to datasets/CASME2/Cropped-updated/Cropped/denoised_masked/sub13/EP02_03/md_reg_img34.jpg
Saving out

In [6]:
print('DONE')

DONE


In [7]:
print('DONE2')

DONE2
